In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_dataportal',
    'd_winthesouth_bu_sku',
    proc_date,
    'temp_d_winthesouth_bu_sku'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_dataportal.d_winthesouth_bu_sku (
  business_unit STRING,
  product_id STRING,
  mien STRING,
  thematic STRING,
  delete BIGINT,
  proc_date DATE,
  upload_user STRING,
  upload_time TIMESTAMP
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
);
""")

In [0]:
source_columns = spark.table("temp_d_winthesouth_bu_sku").columns
source_columns.remove("file_creation_ts")
insert_row = ", ".join(source_columns)

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_silver_dataportal.d_winthesouth_bu_sku
WHERE proc_date = DATE('{proc_date}')
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_silver_dataportal.d_winthesouth_bu_sku ({insert_row})
SELECT {insert_row} FROM temp_d_winthesouth_bu_sku
""")